In [3]:
import os
import cv2
import numpy as np
from pathlib import Path

In [4]:
def draw_mot_tracklets(
    img_folder,
    gt_file,
    output_folder=None,
    show=False,
    box_color_fn=None,
    thickness=2,
    font_scale=0.3
):
    # Read GT file
    tracklets = {}
    with open(gt_file, 'r') as f:
        for line in f:
            if line.strip() == '':
                continue
            frame_id, track_id, x, y, w, h, conf, cls, vis = map(float, line.strip().split(','))
            frame_id = int(frame_id)
            track_id = int(track_id)
            if frame_id not in tracklets:
                tracklets[frame_id] = []
            tracklets[frame_id].append((track_id, int(x), int(y), int(w), int(h)))

    img_names = sorted(os.listdir(img_folder))
    os.makedirs(output_folder, exist_ok=True) if output_folder else None

    for i, img_name in enumerate(img_names, start=1):
        img_path = os.path.join(img_folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        if i in tracklets:
            for tid, x, y, w, h in tracklets[i]:
                color = box_color_fn(tid) if box_color_fn else (0, 255, 0)
                cv2.rectangle(img, (x, y), (x + w, y + h), color, thickness)
                cv2.putText(img, f'{tid}', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                            font_scale, color, thickness=1)

        if output_folder:
            cv2.imwrite(os.path.join(output_folder, img_name), img)

        if show:
            cv2.imshow("Tracking", img)
            if cv2.waitKey(30) == 27:
                break

    if show:
        cv2.destroyAllWindows()

In [12]:
def draw_mot_tracklets_to_video(
    img_folder,
    gt_file,
    output_video_path,
    box_color_fn=None,
    thickness=1,
    font_scale=0.5,
    frame_rate=15
):
    # Load tracking annotations
    tracklets = {}
    with open(gt_file, 'r') as f:
        for line in f:
            if line.strip() == '':
                continue
            frame_id, track_id, x, y, w, h, conf, _, _,_ = map(float, line.strip().split(','))
            frame_id = int(frame_id)
            track_id = int(track_id)
            if frame_id not in tracklets:
                tracklets[frame_id] = []
            tracklets[frame_id].append((track_id, int(x), int(y), int(w), int(h)))

    # Get sorted list of frame image paths
    img_files = sorted([f for f in os.listdir(img_folder) if f.endswith(('.jpg', '.png'))])
    if not img_files:
        raise ValueError("No images found in the folder.")

    # Read first frame to get size
    first_frame = cv2.imread(os.path.join(img_folder, img_files[0]))
    if first_frame is None:
        raise ValueError("Could not read first image.")
    height, width = first_frame.shape[:2]

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID', 'avc1' etc.
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    # Process each frame
    for i, img_name in enumerate(img_files, start=0):
        img_path = os.path.join(img_folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not read {img_path}")
            continue

        if i in tracklets:
            for tid, x, y, w, h in tracklets[i]:
                color = box_color_fn(tid) if box_color_fn else (0, 255, 0)
                cv2.rectangle(img, (x, y), (x + w, y + h), color, thickness)
                cv2.putText(img, f'ID {tid}', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                            font_scale, color, thickness=1)

        video_writer.write(img)

    video_writer.release()
    print(f"Video saved to: {output_video_path}")

In [16]:
# DATA_BASE_PATH = Path("D:\\Dataset\\Football\\test1\\StrongSORT_CustMOT_Split_Connect")
# DATA_BASE_PATH = Path("F:\\Dataset\\test1\\StrongSORT_Tracklets")
DATA_BASE_PATH = Path("F:\\Dataset\\test_merge_videos_tracklets")
# TRACKLETS_FILE = DATA_BASE_PATH / "seq_merged.txt"
# TRACKLETS_FILE = "F:\\Dataset\\test_merge_videos_tracklets\\result_videos\\seq_mapped.txt"
TRACKLETS_FILE = "F:\\Dataset\\test_merge_videos_tracklets\\part1\\outputs\\seq_0.txt"
# IMG_DIR = DATA_BASE_PATH / "merged_images"
# IMG_DIR = "F:\\Dataset\\test_merge_videos_tracklets\\part2\\outputs\\seq_0\\img1"
IMG_DIR = "F:\\Dataset\\test_merge_videos_tracklets\\part1\\outputs\\seq_0\\img1"
OUTPUT_VIDEO_PATH =  "F:\\Dataset\\test_merge_videos_tracklets\\result_videos\\video_0.mp4"

In [17]:
# Example usage
draw_mot_tracklets_to_video(
    img_folder=IMG_DIR,
    gt_file=TRACKLETS_FILE,
    output_video_path=OUTPUT_VIDEO_PATH,
    box_color_fn=lambda tid: tuple(int(c) for c in np.random.RandomState(tid).randint(0, 255, size=3)),
)

Video saved to: F:\Dataset\test_merge_videos_tracklets\result_videos\video_0.mp4
